# Proyecto : Análisis de mercado inmobiliario. Modelos Avanzados

Creado por Vanesa Flores.

## Objetivo

Crear un modelo Machine Learning que, dadas ciertas características de una propiedad, prediga su precio de venta.


### Librerías útiles

In [ ]:
#Importo algunas Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

#Preprocesamiento
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

# Ingeniería de Features

Cargamos los datos usando la librería `Pandas` y observamos sus características:

In [ ]:
df= pd.read_csv('../input/ds-proyecto-01-datos-properati/DS_Proyecto_01_Datos_Properati.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

Consideraremos descartar algunas columnas que no aportan información relevante relacionada al objetivo del proyecto y renombramos otras para un mejor entendimiento: 

In [ ]:
df=df.drop(columns=['lat', 'lon', 'start_date', 'end_date', 'created_on','title', 'description', 'currency','operation_type','l1'])
df.head()

In [ ]:
df= df.rename(columns={'l2':'provincia','l3':'barrio'})
df.head()

Basándonos en lo estudiado en el primer proyecto, se considera tomar los tipos de propiedad que presentan mayor cantidad de anuncios en nuestro dataset: Departamento, Casa y PH, que representa un 97.6% del total de instancias. También nos quedamos con los anuncios en la provincia: Capital Federal la cual contiene el 63.1% de nuestras observaciones. 

In [ ]:
df= df[(df.property_type.isin(['Departamento', 'Casa','PH']) & (df.provincia== 'Capital Federal'))]
df.head()

In [ ]:
df.shape

Ésto nos deja con un muestra que conserva el 62.37% de nuestro dataset original. Observemos inicialmente cómo se correlacionan estas variables:

In [ ]:
#Correlación inicial
corr_0=df.corr()

plt.figure(figsize=(8,8))
ax=sns.heatmap(corr_0, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()

Se tiene que no existe correlaciones elevadas entre las variables independientes y la variable objetivo `Price`. Sin embargo, la mayor correlación la presentan las variables `rooms`, `bedrooms` y `bathrooms`.

# Imputación de datos faltantes

En lo que sigue vamos a observar de forma detallada la presencia de datos faltantes en nuestro dataset.

In [ ]:
faltantes=df.isna().sum()
print('Valores faltantes por columnas:', faltantes)

Calculemos el porcentaje de faltantes en los atributos de Bathrooms, Surface_total y Surface_covered.

In [ ]:
cant_total=df.barrio.count()
porcentaje_Na=(faltantes/cant_total)*100
porcentaje_Na

Tenemos un total de 91485 instancias. Donde las variables `bathrooms`, `surface_total` y `surface_covered` poseen un 2.37%, 9.74% y 10.09% de valores faltantes en nuestro dataset, respectivamente. Considerando que el porcentaje de datos faltantes es bajo se decide eliminar las instancias que los contengan.

Con ésto estaríamos evitando realizar una imputación de nulos que quizás luego pudiese sesgar de alguna manera nuestros datos. Sabiendo que los métodos de imputación se basan en medidas de tendencia central como la media o la mediana que presentan poca robutez.

Sin embargo, con intención de evidenciar lo antes mencionado se planteará una imputación de datos faltantes usando la clase `SimpleImputer` de `scikit-learn`. Para luego, al final del proyecto considerar realizar de nuevo la evaluación de los modelos sin eliminar los datos faltantes y poder establecer una comparación. 

In [ ]:
df_sinNA= df.dropna()
df_sinNA.shape

In [ ]:
df_sinNA.isnull().sum()

In [ ]:
df_sinNA= df_sinNA.reset_index()

* Implementación de la clase `SimpleImputer`:

In [ ]:
#Imputaremos con la media
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
df_imput= df.drop(columns=[ 'provincia', 'barrio','property_type'])
imp_mean.fit(df_imput)

In [ ]:
imp_mean.transform(df_imput)

In [ ]:
df_impt= pd.DataFrame(imp_mean.transform(df_imput), columns=['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered',
       'price'])
df_impt.head()

In [ ]:
df_impt.isnull().sum()

De esta forma tenemos dos datasets, ambos sin datos faltantes, uno con nulos eliminados (`df_sinNA`) y otro con nulos imputados (`df_impt`). En lo que sigue, y basándonos en lo antes expuesto continuaremos con el dataset con los faltantes eliminados. 

# Detección de outliers

Siguiendo con el preprocesamiento de nuestro dataset, veamos ahora cómo es la distribución de la variables realizando boxplots para cada una de ellas:

In [ ]:
df_num= df_sinNA.drop(columns=['index','provincia', 'barrio', 'property_type'])
fig, axs = plt.subplots(6)
fig.suptitle('Distribución de Variables')
for i,el in enumerate(list(df_num.columns.values)[:]):
    a = df_sinNA.boxplot(el, ax=axs.flatten()[i],vert=False)

Se observa presencia de valores atípicos en todas las variables. Éstos parecen encontrarse con mayor peso en las últimas tres donde ni siquiera es posible distinguir una distribución de forma clara.

Para detectar la cantidad de outliers presente en cada variable, se utilizará la técnica del rango intercuartílico. 

### Cálculo de Rango Intercuartílico

In [ ]:
columnas=['rooms','bedrooms','bathrooms','surface_total','surface_covered', 'price']
print('Valores= (q25,q75, IQR, mínimo, máximo)')
for i,columna in enumerate([df_sinNA.rooms,df_sinNA.bedrooms,df_sinNA.bathrooms,df_sinNA.surface_total,df_sinNA.surface_covered,df_sinNA.price]):
    q25,q75 = np.percentile(columna.values, [25,75])
    iqr = q75 - q25
    minimo = q25 - 1.5*iqr
    maximo = q75 + 1.5*iqr
    mascara_outliers = (columna < minimo) | (columna > maximo)
    cant_outliers=mascara_outliers.sum()
    print(f'Valores para {columnas[i]}:',q25,q75,iqr, minimo, maximo)
    print(f'Cantidad de outliers:',cant_outliers)

Podemos observar que existe presencia de valores atípicos en todas nuestras variables. Sin embargo, sólo los eliminaremos para nuestra la variable objetivo, el precio.

In [ ]:
df_sin_out=df_sinNA[(df_sinNA.price > 0 ) & (df_sinNA.price < 512500.0)]

df_sin_out=df_sin_out.reindex(columns=['index','provincia','barrio','rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered', 'price','property_type'])
df_sin_out.drop(['index'], axis=1, inplace=True)
df_sin_out.head()

Observemos ahora la distribución de la variable precio luego de eliminar valores atípicos

In [ ]:
sns.distplot(df_sin_out.price)

In [ ]:
sns.boxplot(df_sin_out.price)

Nos encontramos con una distribución que presenta asimetría positiva que pudiese estar relacionada a propiedades que tengan una superficie grande o que se encuentren en algún barrio que por sus características sea más costoso en la Ciudad. Agrupemos por tipo de propiedad y barrio para observarlo:

In [ ]:
df_sin_out.loc[df_sin_out.price > 400000].groupby(['property_type']).size()

In [ ]:
df_sin_out.loc[df_sin_out.price > 400000].groupby(['barrio']).size()

Los barrios que poseen mayor cantidad de propiedades costosas son los más residenciales y que además, están categorizados como "caros" en la Capital Federal.

# Encoders

Buscaremos generar variables binarias para los atributos `barrio` y `property_type` con el objetivo de incluirlas en los modelos y encontrar mejor desempeño de los mismos. Para esto haremos la codificación de la variable utilizando la funcionalidad `OneHotEncoder` de `Scikit-Learn`.

In [ ]:
ohe = OneHotEncoder(sparse=False).fit_transform(df_sin_out[['barrio','property_type']])
categoricas = pd.DataFrame(ohe)

In [ ]:
categoricas = categoricas.set_index(df_sin_out.index)
df_encoding_mean = pd.concat([df_sin_out, categoricas], axis=1)
df_encoding_mean.head()

In [ ]:
df_encoding_mean.columns=['provincia','barrio','rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered', 'price','property_type',
                                                   'Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
                                                   'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso',
                                                   'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
                                                   'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
                                                   'Parque Patricios', 'Paternal', 'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
                                                   'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
                                                   'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real',
                                                   'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati', 'Villa Urquiza','Villa del Parque','Casa', 
                                                   'Departamento', 'PH']
df_encoding_mean.drop(columns='provincia', inplace=True)
df_encoding_mean.head()

# Escalado de datos

Sabiendo que algunos algoritmos Machine Learning son sensibles a la escala en las que se miden nuestros atributos, procedemos a realizar un escalado de los mismos utilizando la clase `MinMaxScaler` de `scikit-learn` que nos permite obtener atributos escalados en un rango determinado. Esto con la intención de evitar falsos resultados de nuestros modelos más adelante.

En este punto, para facilitar volver después a nuestros datos no escalados se realizará el escalado por separado de las variables independientes y las variables dependientes.

In [ ]:
scaler_independientes= MinMaxScaler(feature_range=(0,1))
scaler_dependientes=MinMaxScaler(feature_range=(0,1))

En lo que sigue, selecciono las variables con las que más adelante trabajaré con mis modelos. 

In [ ]:
#Selecciono variables numéricas
variables_independientes=df_encoding_mean[['rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered',
                                                   'Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
                                                   'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso',
                                                   'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
                                                   'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
                                                   'Parque Patricios', 'Paternal', 'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
                                                   'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
                                                   'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real',
                                                   'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati', 'Villa Urquiza','Villa del Parque','Casa', 
                                                   'Departamento', 'PH']]
variables_dependientes=df_encoding_mean[['price']]

In [ ]:
fit_independientes=scaler_independientes.fit(variables_independientes)
fit_dependientes=scaler_dependientes.fit(variables_dependientes)

In [ ]:
escalados_independientes=scaler_independientes.transform(variables_independientes)
escalados_dependientes=scaler_dependientes.transform(variables_dependientes)

In [ ]:
df_escalados_independientes=pd.DataFrame(escalados_independientes, columns=['rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered',
                                                   'Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
                                                   'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso',
                                                   'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
                                                   'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
                                                   'Parque Patricios', 'Paternal', 'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
                                                   'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
                                                   'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real',
                                                   'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati', 'Villa Urquiza','Villa del Parque','Casa', 
                                                   'Departamento', 'PH'])
df_escalados_dependientes=pd.DataFrame(escalados_dependientes, columns=["price"])

In [ ]:
df_escalados_independientes.head()

In [ ]:
df_escalados = pd.concat([df_escalados_independientes,df_escalados_dependientes], axis=1)
df_escalados.head()

In [ ]:
sns.distplot(df_escalados.price)

Se observa que la distribución de los datos es la misma, y que nuestra variable precio esta contemplada en un rango del 0 al 1. 

# Relación entre variables 

Observemos un poco las características de nuestros atributos:

In [ ]:
df_escalados.describe()

Veamos cómo están correlacionadas nuestras variables 

In [ ]:
#Estudio de correlaciones
plt.figure(figsize = (8*3,8))

corr1=df_escalados[['rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered',
                                                   'Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
                                                   'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
                                                   'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
                                                   'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
                                                   'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real','Casa', 
                                                   'Departamento', 'PH','price']].corr()
plt.subplot(1, 2, 1)
ax=sns.heatmap(corr1, cbar = True,  square = True, annot=False, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 60)
plt.yticks(rotation = 15)

plt.subplot(1, 2, 2)
corr2=df_escalados[['rooms', 'bedrooms', 'bathrooms','surface_total', 'surface_covered', 'Casa','Departamento','PH','price']].corr()
ax=sns.heatmap(corr2, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 45)
plt.yticks(rotation = 15)

plt.tight_layout()
plt.show()

El mapa de calor del lado izquierdo muestra que las variables binarias generadas para los barrios no muestran correlación significativa con la variable target. 

Sin embargo, para el gráfico del lado derecho se filtran las variables a fin observar con mayor claridad las correlaciones. Se obtiene que comparándo con el Proyecto 1, se observa gran diferencia en las correlaciones respecto a la variable objetivo. Ésta variación pudiese estar relacionada a la eliminación de valores atípicos sobre las variables de superficie. 

# Reducción de la Dimensionalidad (PCA)

En lo que sigue, aplicaremos el método de Análisis de Componentes Principales para indagar si a partir de la reducción de atributos y generación de nuevas variables que obtengan la mayor variabilidad, se encontraran mejores resultados en nuestros modelos.

Lo primero que haremos será separar nuestras variables dependientes de las independientes.

In [ ]:
X=df_escalados[['rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered',
                                                   'Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Boca', 'Boedo',
                                                   'Caballito', 'Catalinas', 'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales', 'Congreso',
                                                   'Constitución', 'Flores', 'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat', 'Monte Castro',
                                                   'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
                                                   'Parque Patricios', 'Paternal', 'Pompeya', 'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
                                                   'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield', 'Versalles', 'Villa Crespo', 'Villa Devoto',
                                                   'Villa General Mitre', 'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón', 'Villa Real',
                                                   'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati', 'Villa Urquiza','Villa del Parque','Casa', 
                                                   'Departamento', 'PH']]
y=df_escalados[['price']]

Utilizamos la función `PCA` de `scikit-learn` para calcular las componentes principales. Para empezar, consideraremos graficar la variabilidad explicada respecto al número de componentes calculadas y estableciendo un umbral de corte que nos indique cuántas componentes nos explican el 95% de la varianza. 

In [ ]:
pca = PCA().fit(X)

plt.rcParams["figure.figsize"] = (18,6)

fig, ax = plt.subplots()
xi = np.arange(1, 66, step=1)
yi = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')

plt.xlabel('Número de componentes')
plt.xticks(np.arange(0, 66, step=1), rotation=90) 
plt.ylabel('Varianza acumulada (%)')
plt.title('Número de componentes necesarias para explicar la varianza')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, 'Umbral de corte: 95%', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

Nos quedamos entonces con las primeras 40 componentes principales calculadas.

In [ ]:
pca = PCA(n_components=40)
X_nuevo = pca.fit_transform(X) 

In [ ]:
pca.components_

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

# Comparemos con el Proyecto 1

Luego del preprocesamiento de datos realizado, consideraremos entrenar tres modelos Machine Learning. Un árbol de decisión, KNN y regresión lineal con los mismos parámetros utilizados en el Proyecto 1 con el fin de comparar resultados. 

### Modelo 1: Árbol de Decisión con PCA

Realizamos ahora la división de nuestros datos, considerando un conjunto de datos de entrenamiento y otro de testeo. Es importante resaltar que en este punto nuestros atributos son las dos componentes pricipales calculadas (`X_nuevo`).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y, test_size=0.30, random_state=42)

In [ ]:
tree_regressor = DecisionTreeRegressor(max_depth=8, random_state=42)
tree_regressor.fit(X_train, y_train)

Definiremos una función que nos permitirá evaluar cada modelo de manera más fácil y además, nos mostrará los errores obtenidos mediante el cálculo del RMSE con las valores en la escala real y el valor de r2_score que nos indica el desempeño del modelo.

In [ ]:
def evaluar_modelo(model, scaler, X_train, X_test, y_train, y_test):
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    ### VOLVEMOS A LAS VARIABLES NO ESCALADAS
    y_train_pred_desesc = scaler.inverse_transform(y_train_pred.reshape(-1, 1))
    y_test_pred_desesc = scaler.inverse_transform(y_test_pred.reshape(-1, 1))
    
    y_train_desesc = scaler.inverse_transform(y_train)
    y_test_desesc = scaler.inverse_transform(y_test)
    
    ### CALCULAMOS EL ERROR
    rmse_train = np.sqrt(mean_squared_error(y_train_desesc, y_train_pred_desesc))
    rmse_test = np.sqrt(mean_squared_error(y_test_desesc, y_test_pred_desesc))

    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    r2=r2_score(y_true=y_test_desesc, y_pred=y_test_pred_desesc)
    print(f'El valor del r2 score : {r2}')
    
    plt.figure(figsize = (15,4))

    plt.subplot(1,2,1)
    sns.distplot(y_train_desesc - y_train_pred_desesc, bins = 20, label = 'train')
    sns.distplot(y_test_desesc - y_test_pred_desesc, bins = 20, label = 'test')
    plt.xlabel('errores')
    plt.legend()


    ax = plt.subplot(1,2,2)
    ax.scatter(y_test_desesc,y_test_pred_desesc, s =2)
   
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
      ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
   
    plt.tight_layout()
    plt.show()

In [ ]:
evaluar_modelo(tree_regressor, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 2: KNN con PCA

In [ ]:
knn_regressor = KNeighborsRegressor(n_neighbors=10)
knn_regressor.fit(X_train, y_train)
evaluar_modelo(knn_regressor, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 3: Regresión Lineal con PCA

In [ ]:
linear_model = LinearRegression() #Benchmark
linear_model.fit(X_train, y_train)
evaluar_modelo(linear_model, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
metricas= [[57286.50, 57958.02,-0.02, 65750.65, 64776.66, 0.56],
         [46297.29, 47935.92, 0.48, 55369.93,55594.84,0.68],
         [ 41839.18, 45999.04,0.57,39261.32,42334.09,0.81]]

columnas = ['RMSE_Train_P1', 'RMSE_Test_P1', 'R2_score_P1', 'RMSE_Train P2_PCA', 'RMSE_Test_P2_PCA', 'R2_score_P2_PCA'] # definimos los nombres de las columnas
filas = ['Regresión lineal', 'Árbol de Decisión', 'KNN'] # definimos los nombres de las filas

comparacion = pd.DataFrame(metricas, columns=columnas, index=filas)
comparacion.head()

Se puede observar que el performance de los tres modelos indicados mejoran su desempeño respecto a los resultados obtenidos en el Proyecto 1, considerando los mismos hiperparámetros. Podemos resaltar que el modelo de vecinos más cercanos arroja el r2_score más alto y además, presenta similitud en la distribuciones de los errores de train y test, por lo que podemos inferir que no existe un sobreajuste significativo del modelo. 

Veamos ahora, cómo es el desempeño de nuestro modelos sin usar las componentes principales calculadas (PCA) y en cambio, usando sólo las variables predictoras que presentan mayor correlación con la variable objetivo.

In [ ]:
X=df_escalados[['rooms','bedrooms','bathrooms']]
y=df_escalados[['price']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### Modelo 4: Árbol de Decisión (Variables mayor corr)

In [ ]:
tree_regressor1 = DecisionTreeRegressor(max_depth=8, random_state=42)
tree_regressor1.fit(X_train, y_train)
evaluar_modelo(tree_regressor1, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 5: KNN (Variables mayor corr)

In [ ]:
knn_regressor1 = KNeighborsRegressor(n_neighbors=10)
knn_regressor1.fit(X_train, y_train)
evaluar_modelo(knn_regressor1, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 6: Regresión Lineal (Variables mayor corr)

In [ ]:
linear_model1 = LinearRegression() #Benchmark
linear_model1.fit(X_train, y_train)
evaluar_modelo(linear_model1, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
metricas= [[57286.50, 57958.02,-0.02, 65750.65, 64776.66, 0.56,72248.10, 71127.17,0.47],
         [46297.29, 47935.92, 0.48, 55369.93,55594.84,0.68, 68869.13, 68466.13, 0.51],
         [ 41839.18, 45999.04,0.57,39261.32,42334.09,0.81, 71935.27, 71387.44,  0.47]]

columnas = ['RMSE_Train_P1', 'RMSE_Test_P1', 'R2_score_P1', 'RMSE_Train P2_PCA', 'RMSE_Test_P2_PCA', 'R2_score_P2_PCA', 'RMSE_Train_var_corr', 'RMSE_Test_var_corr', 
            'R2_score_var_corr'] # definimos los nombres de las columnas
filas = ['Regresión lineal', 'Árbol de Decisión', 'KNN'] # definimos los nombres de las filas

comparacion = pd.DataFrame(metricas, columns=columnas, index=filas)
comparacion.head()

De esta primera parte en donde hemos considerado evaluar los modelos con hiperparámetros definidos e iguales para cada iteración de los modelos, se obtiene:

- Realizar la reducción de dimensionalidad mediante el análisis de componentes principales nos ofrece mejoras significativas a nuestros modelos.

- El modelo que mejor performance arroja hasta ahora es el KNN.

- El modelo de regresión lineal mejora en relación a lo obtenido el el proyecto anterior, sin embargo, es el modelo que peor performance arroja.

- Se resalta que la distribución de densidad para los errores de los conjuntos de train y test no difieren significativamente en cada modelo. Además, los histogramas son bastantes simétricos y parecidos para ambos conjuntos.

# PARTE B.  Optimización de Hiperparámetros

En lo que sigue, se intentará optimizar los modelos anteriores haciendo uso de la funcionalidad `RandomizedSearchCV` de `scikit-learn`, la cual nos permite hacer variaciones de los hiperparámetros del modelo de forma aleatoria y además incluye la estrategia de Validación Cruzada para obtener independencia de la partición de nuestros datos.

Empecemos haciendo el entrenamiento de los modelos considerando la variables que presentan más correlación con nuestra variable objetivo. En esta parte sólo consideraremos los modelos árbol de decisión y knn ya que más adelante se propone realizar regresión con regularización y atributos polinómicos.

In [ ]:
X=df_escalados[['rooms','bedrooms','bathrooms']]
y=df_escalados[['price']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### Modelo 7: Árbol de Decisión CV (Variables mayor corr) 

In [ ]:
tree_regressor2 = DecisionTreeRegressor(random_state=42)

Construyamos ahora la grilla de hiperparámetros los cuales serán generados de forma aleatoria para nuestro random search:

In [ ]:
# Grilla para Random Search
param_dist = {'max_depth':np.arange(1,30),                #Profundidad del árbol
              'min_samples_split': [2,3,4],        #El número mínimo de muestras necesarias para dividir un nodo interno.
              }

Busquemos optimizar parámetros

In [ ]:
random_search = RandomizedSearchCV(tree_regressor2, param_dist,n_iter=20, random_state=42, cv=5)
random_search.fit(X_train, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

Reentrenamos con los mejores parámetros:

In [ ]:
tree_regressor3 = DecisionTreeRegressor(max_depth=23, min_samples_split=4, random_state=42)
tree_regressor3.fit(X_train, y_train)
evaluar_modelo(tree_regressor3, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 8: KNN CV (Variables mayor corr)

In [ ]:
knn_regressor2 = KNeighborsRegressor()

In [ ]:
#Random Search
param_dist = {'n_neighbors':np.arange(10,30),     #Número de vecinos
              'weights': ['uniform', 'distance']}        #Función de peso (cómo se ponderan los scores)
random_search = RandomizedSearchCV(knn_regressor2, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
knn_regressor3 = KNeighborsRegressor(n_neighbors=29, weights='distance')
knn_regressor3.fit(X_train, y_train)
evaluar_modelo(knn_regressor3, scaler_dependientes,X_train, X_test, y_train, y_test)

Ahora, veamos el performance de los modelos considerando utilizar la variables generadas con Componentes Principales:

In [ ]:
X=X_nuevo
y=df_escalados[['price']]
#Realizamos la partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### Modelo 9: Árbol de Decisión CV y PCA 

In [ ]:
tree_regressor4 = DecisionTreeRegressor(random_state=42)

In [ ]:
#Random Search
param_dist = {'max_depth':np.arange(1,80),                #Profundidad del árbol
              'min_samples_split': [2,3,4],        #El número mínimo de muestras necesarias para dividir un nodo interno.
              }
random_search = RandomizedSearchCV(tree_regressor4, param_dist,n_iter=20, random_state=42, cv=5)
random_search.fit(X_train, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
tree_regressor5 = DecisionTreeRegressor(max_depth=39, min_samples_split=3, random_state=42)
tree_regressor5.fit(X_train, y_train)
evaluar_modelo(tree_regressor5, scaler_dependientes,X_train, X_test, y_train, y_test)

### Modelo 10: KNN CV y PCA

In [ ]:
knn_regressor4 = KNeighborsRegressor()

In [ ]:
#Random Search
param_dist = {'n_neighbors':np.arange(10,100),     #Número de vecinos
              'weights': ['uniform', 'distance']}        #Función de peso (cómo se ponderan los scores)
random_search = RandomizedSearchCV(knn_regressor4, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
knn_regressor5 = KNeighborsRegressor(n_neighbors=19, weights='distance')
knn_regressor5.fit(X_train, y_train)
evaluar_modelo(knn_regressor5, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
metricas1= [[68734.79, 68479.18, 0.51, 41058.51, 49137.44, 0.75],
         [69899.82, 69611.44, 0.50,18377.27,39262.54,0.84]]

columnas1 = ['RMSE_Train_var_corr', 'RMSE_Test_var_corr', 'R2_score_var_corr','RMSE_Train_PCA', 'RMSE_Test_PCA', 'R2_score_PCA'] # definimos los nombres de las columnas
filas1 = ['Árbol de Decisión', 'KNN'] # definimos los nombres de las filas

comparacion1 = pd.DataFrame(metricas1, columns=columnas1, index=filas1)
comparacion1.head()

En la tabla comparativa se observa:
    
- Existe una pequeña mejoría en el desempeño de ambos modelos considerando las variables con mayor correlación después de la optimización de los hiperparámetros.

- Se encuentran distribuciones de densidad de errores que difieren bastante entre el conjunto de train y test, indicando así la presencia de un sobreajuste en nuestros modelos.  


# Modelos Avanzados

## Random Forest

Veamos ahora el desempeño de un modelo Random Forest entrenado tanto con la variables de mayor correlación con le precio como con las componentes principales.

En principio, entrenaremos el modelo con hiperparámetros puestos de forma manual para observar su desempeño. Luego, realizaremos la validación de hiperparámetros soportándonos en el error Out-of-Bag. Calcularemos los errores de entrenamiento y los oob_scores de forma iterativa variando la cantidad de estimadores en el modelo. De esta manera, podemos obtener cuántos árboles debe modelar el random forest para obtener el mejor performance.

No se realizará el método de optimización de parámetros por Validación Cruzada ya que presenta un costo computacional elevado.

### Modelo 11: Random Forest con PCA

In [ ]:
X=X_nuevo
y=df_escalados[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
random_forest = RandomForestRegressor(
            n_estimators = 10,
            criterion    = 'mse',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,               #Con -1 se utilizan todos los cores disponibles
            random_state = 42
         )
random_forest.fit(X_train, y_train)
evaluar_modelo(random_forest, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
# Validación con Out-of-Bag error
train_scores = []
oob_scores   = []

# Número de estimadores a considerar
estimator_range = range(1, 50, 5)

# Iteramos para distintos valores de n_estimators
for n_estimators in estimator_range:
    random_forest1 = RandomForestRegressor(
                n_estimators = n_estimators,
                criterion    = 'mse',
                max_depth    = None,
                max_features = 'auto',
                oob_score    = True,
                n_jobs       = -1,
                random_state = 42
             )
    random_forest1.fit(X_train, y_train)
    train_scores.append(random_forest1.score(X_train, y_train))
    oob_scores.append(random_forest1.oob_score_)
    
# Graficamos el comportamiento de los errores train y obb_score respecto al número de estimadores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, oob_scores, label="out-of-bag scores")
ax.plot(estimator_range[np.argmax(oob_scores)], max(oob_scores),
        marker='o', color = "red", label="max score")
ax.set_ylabel("R^2")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del out-of-bag-error vs número árboles")
plt.legend();
#Imprimimos el valor óptimo 
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmax(oob_scores)]}")

Se observa que aproximadamente a partir de los 10 estimadores el error obb_score del modelo se estabiliza y se vuelve constante, llegando así a su valor más elevado. Por lo que ahora entrenaremos de nuevo modelo con el valor óptimo obtenido.

In [ ]:
random_forest2 = RandomForestRegressor(
            n_estimators = 46,
            criterion    = 'mse',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,               #Con -1 se utilizan todos los cores disponibles
            random_state = 42
         )
random_forest2.fit(X_train, y_train)
evaluar_modelo(random_forest2, scaler_dependientes,X_train, X_test, y_train, y_test)

Observemos la importancia de los features:

In [ ]:
df_X_nuevo=pd.DataFrame(X_nuevo)
sns.barplot(df_X_nuevo.columns, random_forest2.feature_importances_)
plt.xlabel("Componentes Principales")
plt.ylabel("Importancias")

Se puede observar que sólo 4 componentes principales (14,24,34,35) son las que aportan al modelo una importancia mayor al 5%. Se esperaba que las primeras componentes que respresentaban mayor variabilidad de los datos fuesen las mejores variables predictoras para el modelo. Sin embargo, por ser estas componentes generadas al condensar la información de las variables originales, es difícil interpretar qué representan además de ser las más relevantes del modelo. 

### Modelo 11: Random Forest con variables más corr

In [ ]:
X=df_escalados[['rooms','bedrooms','bathrooms']]
y=df_escalados[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
random_forest3 = RandomForestRegressor(
            n_estimators = 10,
            criterion    = 'mse',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,               #Con -1 se utilizan todos los cores disponibles
            random_state = 42
         )
random_forest3.fit(X_train, y_train)
evaluar_modelo(random_forest3, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
# Validación con Out-of-Bag error
train_scores = []
oob_scores   = []

# Número de estimadores a considerar
estimator_range = range(1, 50, 5)

# Iteramos para distintos valores de n_estimators
for n_estimators in estimator_range:
    random_forest4 = RandomForestRegressor(
                n_estimators = n_estimators,
                criterion    = 'mse',
                max_depth    = None,
                max_features = 'auto',
                oob_score    = True,
                n_jobs       = -1,
                random_state = 42
             )
    random_forest4.fit(X_train, y_train)
    train_scores.append(random_forest4.score(X_train, y_train))
    oob_scores.append(random_forest4.oob_score_)
    
# Graficamos el comportamiento de los errores train y obb_score respecto al número de estimadores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, oob_scores, label="out-of-bag scores")
ax.plot(estimator_range[np.argmax(oob_scores)], max(oob_scores),
        marker='o', color = "red", label="max score")
ax.set_ylabel("R^2")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del out-of-bag-error vs número árboles")
plt.legend();
#Imprimimos el valor óptimo 
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmax(oob_scores)]}")

Se observa nuevamente que aproximadamente a partir de los 10 estimadores el error obb_score del modelo se estabiliza y se vuelve constante, llegando así a su valor más elevado (n_estimators: 41)

In [ ]:
random_forest5 = RandomForestRegressor(
            n_estimators = 41,
            criterion    = 'mse',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,               #Con -1 se utilizan todos los cores disponibles
            random_state = 42
         )
random_forest5.fit(X_train, y_train)
evaluar_modelo(random_forest5, scaler_dependientes,X_train, X_test, y_train, y_test)

In [ ]:
sns.barplot(X.columns, random_forest5.feature_importances_)
plt.xlabel("Componentes Principales")
plt.ylabel("Importancias")

En este caso, el modelo se está entrenando con las variables originales y se obtiene que el atributo que representa la mayor importancia para el modelo es 'bathrooms'. Esto se esperaba ya que es la variable que presenta mayor correlación con la variable a predecir 'price'. 

In [ ]:
metricas2= [[40082.58, 46003.66, 0.78],
         [68739.02, 68422.15, 0.51]]


columnas2 = ['RMSE_Train', 'RMSE_Test', 'R2_score',] # definimos los nombres de las columnas
filas2 = ['Random_Forest_PCA', 'Random_Forest_var_corr'] # definimos los nombres de las filas

comparacion2 = pd.DataFrame(metricas2, columns=columnas2, index=filas2)
comparacion2.head()

Random Forest con componentes principales representa hasta ahora el mejor score y la distribución del error global para el conjunto de train y test con mayor similitud, indicando que no hay existencia de sobreajuste significativa. Además el histograma de errores es bastante simétrico.

## Regresión con Regularización y Atributos Polinómicos

Como hemos visto en la parte A del proyecto, el modelo de regresión lineal es el que peor performance ha arrojado. Esto pudiese estar ligado a la falta de linealidad en nuestro problema. Pensando en esto, consideremos ahora entrenar un modelo de regresión agregando atributos polinómicos usando la función `PolynomialFeatures` de `Scikit-Learn`. 

Además, entrenaremos los modelos aplicandos dos tipos de regularización; la de Ridge y la de Lasso. También, se realizará optimización del parámetro `alpha` que es el que penaliza nuestra función de costo.

### Modelo 12: Regresión Avanzada con PCA

In [ ]:
X=X_nuevo
y=df_escalados[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
poly = PolynomialFeatures(2, include_bias=False)
X_train_new1 = poly.fit_transform(X_train)
X_test_new1 = poly.fit_transform(X_test)
print(X_train_new1.shape, X_test_new1.shape)

### Regularización de Ridge:

In [ ]:
reg_ridge = Ridge()

#Random Search
param_dist = {'alpha':np.logspace(-10, 2, 100),   #Esta funcionalidad de numpy nos genera 100 muestras desde 1e-10 a 1e+2, con espaciado logaritmico.
             }       
random_search = RandomizedSearchCV(reg_ridge, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train_new1, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
reg_ridge1 = Ridge(alpha=0.030538555088334123)
reg_ridge1.fit(X_train_new1,y_train)
evaluar_modelo(reg_ridge1, scaler_dependientes,X_train_new1, X_test_new1, y_train, y_test)

### Regularización de Lasso:

In [ ]:
reg_lasso = Lasso()

#Random Search
param_dist = {'alpha':np.logspace(-10, 2, 100),   
             }       
random_search = RandomizedSearchCV(reg_lasso, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train_new1, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
reg_lasso1 = Lasso(alpha = 5.336699231206313e-06)
reg_lasso1.fit(X_train_new1,y_train)
evaluar_modelo(reg_lasso1, scaler_dependientes,X_train_new1, X_test_new1, y_train, y_test)

### Modelo 13: Regresión Avanzada con variables más corr

In [ ]:
X=df_escalados[['rooms','bedrooms','bathrooms']]
y=df_escalados[['price']]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)

In [ ]:
poly = PolynomialFeatures(3, include_bias=False)
X_train_new2 = poly.fit_transform(X_train)
X_test_new2 = poly.fit_transform(X_test)
print(X_train_new2.shape, X_test_new2.shape)

### Regularización de Ridge:

In [ ]:
reg_ridge2 = Ridge()

#Random Search
param_dist = {'alpha':np.logspace(-10, 2, 200),   
             }       
random_search = RandomizedSearchCV(reg_ridge2, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train_new2, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
reg_ridge3 = Ridge(alpha= 0.005231099308056258)
reg_ridge3.fit(X_train_new2,y_train)
evaluar_modelo(reg_ridge3, scaler_dependientes,X_train_new2, X_test_new2, y_train, y_test)

### Regularización de Lasso:

In [ ]:
reg_lasso2 = Lasso()

#Random Search
param_dist = {'alpha':np.logspace(-10, 2, 100),   
             }       
random_search = RandomizedSearchCV(reg_lasso2, param_dist,n_iter=10, random_state=42, cv=5)
random_search.fit(X_train_new2, y_train)
print("Mejores parametros: "+str(random_search.best_params_))
print("Mejor Score: "+str(random_search.best_score_)+'\n')

In [ ]:
reg_lasso3 = Lasso(alpha = 4.641588833612782e-08)
reg_lasso3.fit(X_train_new2,y_train)
evaluar_modelo(reg_lasso3, scaler_dependientes,X_train_new2, X_test_new2, y_train, y_test)

In [ ]:
metricas3= [[58194.57, 57439.64, 0.66,69594.88, 68981.78, 0.51],
            [ 59466.23, 58648.54, 0.64, 69591.36, 68975.65, 0.51]]

columnas3 = ['RMSE_Train_PCA', 'RMSE_Test_PCA', 'R2_score_PCA','RMSE_Train_var_corr','RMSE_Test_var_corr','R2_score_var_corr'] # definimos los nombres de las columnas
filas3 = ['Regularización_Ridge', 'Regularización_Lasso'] # definimos los nombres de las filas

comparacion3 = pd.DataFrame(metricas3, columns=columnas3, index=filas3)
comparacion3.head()

Podemos observar que existen mejoras en el modelo de Regresión, considerando atributos polinómicos y regularización de Ridge y de Lasso. Sin embargo, sigue teniendo un performance bastante bajo y con errores muy elevados.

- Para los modelos de regresión avanzada considerados, el mejor performance lo arrojan los entrenados con las componentes principales.
- Es importante destacar que la regularización de Lasso arroja una distribución de densidad de error muy similar para los conjuntos de train y test, además de histogramas simétricos, por lo que si bien no es modelo con mejor r2 score, es el que no presenta sobreajuste.
- La inclusión de atributos polinómicos aporta gran mejoría en los modelos de regresión con regularización respecto a un modelo de regresión lineal benchmark como el considerado en la parte A del proyecto. 

In [ ]:
#Tabla final
metricas4= [[65750.65, 64776.66, 0.56,72248.10, 71127.17,0.47],
         [55369.93,55594.84,0.68, 68869.13, 68466.13, 0.51],
         [39261.32,42334.09,0.81, 71935.27, 71387.44,  0.47],
          [41058.51, 49137.44, 0.75,68734.79, 68479.18, 0.51],
         [18377.27,39262.54,0.84,69899.82, 69611.44, 0.50],
          [40082.58, 46003.66, 0.78, 68739.02, 68422.15, 0.51],
          [58194.57, 57439.64, 0.66,69594.88, 68981.78, 0.51],
            [ 59466.23, 58648.54, 0.64, 69591.36, 68975.65, 0.51]]

columnas4 = ['RMSE_Train_PCA', 'RMSE_Test_PCA', 'R2_score_PCA', 'RMSE_Train_var_corr', 'RMSE_Test_var_corr', 
            'R2_score_var_corr'] # definimos los nombres de las columnas
filas4 = ['Regresión lineal', 'Árbol de Decisión', 'KNN','Árbol de Decisión CV', 'KNN CV', 'Random Forest', 'Regresión Reg Ridge', 'Regresión Reg Lasso'] # definimos los nombres de las filas

comparacion4 = pd.DataFrame(metricas4, columns=columnas4, index=filas4)

# Conclusiones

A lo largo de todo el proyecto se ha trabajado de forma iterativa para encontrar el modelo que mejor prediga el precio de una propiedad en base a sus características. Comparando el desempeño de los modelos tomando en cuenta las componentes principales que aportan mayor variabilidad al problema y también, considerando usar las variables predictoras que presentan mayor correlación con la variable objtivo.

En la siguiente tabla recolectamos todos los resultados:

In [ ]:
comparacion4

* El modelo de vecinos más cercanos arroja un R2 alto, pero presenta overfitting ya que los errores de train y test difieren bastante, lo que indica que el modelo está aprendiendo demasiado de los datos pero esta prediciendo de forma fallida. Es importante resaltar que KNN presenta menor overfitting cuando se considera estimar los pesos de forma uniforme que con el inverso de la distancia.

* El mejor modelo entrenado con nuestros datos es el Random Forest, arroja los errores más bajos, con distribuciones de errores de train y test menor sobreajuste y un R2 score de 0.78. Lo cual indica que estamos aproximando nuestros datos de buena manera. 

* Los modelos regresión mejoran con la transformación de los datos, la inclusión de atributos polinómicos y regularización con una variación considerable del parámetro alpha. El modelo de regularización de Lasso presenta una muy buena distribución de errores para los conjuntos de train y test, además el gráfico de valores predichos y valores reales presentan menor dispersión y se ajustan más a la recta, lo cual indica que es un buen regresor.

* Se resalta la importancia de la reducción de la dimensionalidad. Los modelos de árboles, vecinos y random forest; son los que presentan mejor performance con estas variables.

Considero que se podría mejorar el desempeño de los modelos prestando atención a las variables de superficie total y cubierta. Evaluar con los expertos el tema de si éstas variables tienen un distribución común o están sesgadas de alguna forma y por esto presentan valores atípicos que podamos luego remover con mayor certeza.

# Clustering Prueba (Extra)

Después de algunas iteraciones encontré que agregando las variables binarias generadas a partir de la variable property_type, la el diagrama de dispersión de las dos primeras componentes principales muestran cierto patrón de agrupamiento, tal como se muestra a continuación:

In [ ]:
X=df_escalados[['Casa','Departamento','PH','rooms','bedrooms','bathrooms', 'surface_total', 'surface_covered']]
y=df_escalados[['price']]

In [ ]:
pca = PCA(n_components=2)
X_nuevo = pca.fit_transform(X) 

In [ ]:
pca.components_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.scatter(X_nuevo[:,0], X_nuevo[:,1] )
plt.xlabel('PC1')
plt.ylabel('PC2')

Se observan 3 grupos bien definidos. Apliquemos ahora la técnica de K-means

In [ ]:
# Preparamos una lista donde vamos a ir agregando las distancias medias
Sum_of_squared_distances = []
# Fiteammos un modelo para cada numero de cluster que queremos testear
K = np.arange(2,14)
for k in K:
    # Definimos y entrenamos el modelo
    km = KMeans(n_clusters=k)
    km = km.fit(X_nuevo)
    # Calculamos la distancia media y agregamos a la lista
    distancia_total = km.inertia_
    distancia_media = np.divide(distancia_total,X_nuevo.shape[0]) #OJO
    Sum_of_squared_distances.append(distancia_media)
    
plt.figure(figsize = (10,7))
plt.plot(K, Sum_of_squared_distances, lw=3)
plt.scatter(K, Sum_of_squared_distances,s=55,c='r')
plt.xlabel('Cantidad de Clusters K')
plt.ylabel('Inercia media')
plt.title('Método del codo')

plt.show()

Se evidencia de forma clara que el número de clusters idóneo es k=3, ahora entrenemos el modelo:

In [ ]:
# Defino y entreno el modelo
km = KMeans(n_clusters=3)
km = km.fit(X_nuevo)
# Obtengo la posición de los centros y las etiquetas
etiquetas_ = km.labels_
centros_ = km.cluster_centers_
# Plotting the cluster centers and the data points on a 2D plane
sns.scatterplot(X_nuevo[:,0], X_nuevo[:,1], hue = etiquetas_)
sns.scatterplot(centros_[:, 0], centros_[:, 1],color='black', marker="+", s=1000)
plt.title('Data points and cluster centroids')
plt.show()

Si bien el objetivo del proyecto es predecir el precio de una propiedad a partir de sus características, acá podemos evidenciar un modelo que nos permita clasificar propiedades de acuerdo a su tipo. En este caso, los clusters encontrados probablemente están relacionados con el hecho de si una propiedad es de tipo Casa, Departamento o PH.